<a href="https://colab.research.google.com/github/64RK/Nexflix-Project/blob/main/Nexflix_Project.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install scikit-surprise

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

import math
import matplotlib.pyplot as plt
from surprise import Reader, Dataset, SVD
from surprise.model_selection import cross_validate

In [ ]:
from google.colab import drive
drive.mount('/content/drive')
%cd '/content/drive/My Drive/Colab_Experiment_Folder'

dataset = pd.read_csv('combined_data_1.txt',header = None, names = ['Cust_Id', 'Rating'], usecols = [0,1])
dataset['Rating'] = dataset['Rating'].astype(float)

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
/content/drive/My Drive/Colab_Experiment_Folder


In [ ]:
from google.colab import files
df_title = files.upload()

df_title = pd.read_csv('movie_titles.csv', encoding = "ISO-8859-1", header = None, names = ['Movie_Id', 'Year', 'Name'])
df_title.set_index('Movie_Id', inplace = True)
print (df_title.head(10))

Saving movie_titles.csv to movie_titles.csv
            Year                          Name
Movie_Id                                      
1         2003.0               Dinosaur Planet
2         2004.0    Isle of Man TT 2004 Review
3         1997.0                     Character
4         1994.0  Paula Abdul's Get Up & Dance
5         2004.0      The Rise and Fall of ECW
6         1997.0                          Sick
7         1992.0                         8 Man
8         2004.0    What the #$*! Do We Know!?
9         1991.0      Class of Nuke 'Em High 2
10        2001.0                       Fighter


In [ ]:
# To reset the index of the dataframe
df_nan = pd.DataFrame(pd.isnull(dataset.Rating))
df_nan = df_nan[df_nan['Rating'] == True]
df_nan = df_nan.reset_index()

df_nan.head()

,index,Rating
0,0,True
1,548,True
2,694,True
3,2707,True
4,2850,True


In [ ]:
#To create a numpy array containing movie ids according the 'ratings' dataset

movie_np = []
movie_id = 1
#into tuple
for i,j in zip(df_nan['index'][1:],df_nan['index'][:-1]):
    # numpy approach
    temp = np.full((1,i-j-1), movie_id)
    movie_np = np.append(movie_np, temp)
    movie_id += 1

# Account for last record and corresponding length
# numpy approach
last_record = np.full((1,len(dataset) - df_nan.iloc[-1, 0] - 1),movie_id)
movie_np = np.append(movie_np, last_record)

print(f'Movie numpy: {movie_np}')
print(f'Length: {len(movie_np)}')

Movie numpy: [1.000e+00 1.000e+00 1.000e+00 ... 4.499e+03 4.499e+03 4.499e+03]
Length: 24053764


In [ ]:
#To append the above created array to the datset after removing the 'nan' rows
dataset = dataset[pd.notnull(dataset['Rating'])]#movies count so nan
dataset['Movie_Id'] = movie_np.astype(int)
dataset['Cust_Id'] =dataset['Cust_Id'].astype(int)
print('-Dataset examples-')
dataset.head()

<ipython-input-34-949fa75c1e41>:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  dataset['Movie_Id'] = movie_np.astype(int)


-Dataset examples-


,Cust_Id,Rating,Movie_Id
1,1488844,3.0,1
2,822109,5.0,1
3,885013,4.0,1
4,30878,4.0,1
5,823519,3.0,1


In [ ]:
f = ['count','mean']
dataset_movie_summary = dataset.groupby('Movie_Id')['Rating'].agg(f)
dataset_movie_summary.index = dataset_movie_summary.index.map(int)
movie_benchmark = round(dataset_movie_summary['count'].quantile(0.7),0)
drop_movie_list = dataset_movie_summary[dataset_movie_summary['count'] < movie_benchmark].index
print('Movie minimum times of review: {}'.format(movie_benchmark))

Movie minimum times of review: 1799.0


In [ ]:
dataset_cust_summary = dataset.groupby('Cust_Id')['Rating'].agg(f)
dataset_cust_summary.index = dataset_cust_summary.index.map(int)
cust_benchmark = round(dataset_cust_summary['count'].quantile(0.7),0)
drop_cust_list = dataset_cust_summary[dataset_cust_summary['count'] < cust_benchmark].index

print(f'Customer minimum times of review: {cust_benchmark}')

Customer minimum times of review: 52.0


In [ ]:

print('-Data Examples-')
dataset.head()

-Data Examples-


,Cust_Id,Rating,Movie_Id
1,1488844,3.0,1
2,822109,5.0,1
3,885013,4.0,1
4,30878,4.0,1
5,823519,3.0,1


In [ ]:
reader = Reader()
data = Dataset.load_from_df(dataset[['Cust_Id', 'Movie_Id', 'Rating']][:100000], reader)
svd = SVD()
cross_validate(svd, data, measures=['RMSE', 'MAE'], cv=3, verbose=True)

Evaluating RMSE, MAE of algorithm SVD on 3 split(s).

                  Fold 1  Fold 2  Fold 3  Mean    Std     
RMSE (testset)    1.0475  1.0504  1.0555  1.0512  0.0033  
MAE (testset)     0.8358  0.8416  0.8452  0.8409  0.0039  
Fit time          1.06    0.87    0.93    0.95    0.08    
Test time         0.12    0.26    0.16    0.18    0.06    


{'test_rmse': array([1.04754497, 1.05040725, 1.05555   ]),
 'test_mae': array([0.83578814, 0.84160968, 0.84524887]),
 'fit_time': (1.0615570545196533, 0.8654439449310303, 0.9252192974090576),
 'test_time': (0.12455558776855469, 0.2586479187011719, 0.16375279426574707)}

In [ ]:
dataset.head()

,Cust_Id,Rating,Movie_Id
1,1488844,3.0,1
2,822109,5.0,1
3,885013,4.0,1
4,30878,4.0,1
5,823519,3.0,1


In [ ]:
# 1. Find out the list of most popular and liked genre

p = pd.DataFrame(dataset.groupby('Movie_Id')['Rating'].mean())
print("most popular and liked genre", p.sort_values(by= 'Rating', ascending=False).head(5))

most popular and liked genre             Rating
Movie_Id          
3456      4.670989
3033      4.586364
2102      4.581296
4238      4.554434
13        4.552000


In [ ]:
# 2. Create Model that finds the best suited Movie for one user in every genre.

# Create a shallow copy for the movies dataset
user_71264 = df_title.copy()
user_71264 = user_71264.reset_index()
user_71264 = user_71264[~user_71264['Movie_Id'].isin(drop_movie_list)]


# getting full dataset
dataset_71264 = dataset[dataset['Cust_Id'] == 71264]
data = Dataset.load_from_df(dataset_71264[['Cust_Id', 'Movie_Id', 'Rating']], reader)

#create a training set for svd
trainset = data.build_full_trainset()
svd.fit(trainset)

#Predict the ratings
user_71264['Estimate_Score'] = user_71264['Movie_Id'].apply(lambda x: svd.predict(712664, x).est)
user_71264 = user_71264.drop('Movie_Id', axis = 1)
user_71264 = user_71264.sort_values('Estimate_Score', ascending=False)

#Print top 10 recommendations
print(user_71264.head(10))

         Year                   Name  Estimate_Score
2      1997.0              Character               5
12888  1975.0    Race with the Devil               5
12890  2003.0           24: Season 3               5
12891  1995.0                  Smoke               5
12892  2004.0   Driving Miss Wealthy               5
12893  1960.0       The Time Machine               5
12894  1997.0                Ivanhoe               5
12895  1994.0  Bullets Over Broadway               5
12896  2002.0            Unspeakable               5
12897  1989.0   Fright Night Part II               5


/usr/local/lib/python3.10/dist-packages/numpy/core/fromnumeric.py:3432: RuntimeWarning: Mean of empty slice.
  return _methods._mean(a, axis=axis, dtype=dtype,
/usr/local/lib/python3.10/dist-packages/numpy/core/_methods.py:190: RuntimeWarning: invalid value encountered in double_scalars
  ret = ret.dtype.type(ret / rcount)


In [ ]:
# 3. Find what Genre Movies have received the best and worst ratings based on User Rating

p = pd.DataFrame(dataset.groupby('Movie_Id')['Rating'].mean())
print("Movies which have received the best ratings", p.sort_values(by= 'Rating', ascending=False).head(5))
print("Movies which have received the worst ratings", p.sort_values(by= 'Rating', ascending=True).head(5))

Movies which have received the best ratings             Rating
Movie_Id          
3456      4.670989
3033      4.586364
2102      4.581296
4238      4.554434
13        4.552000
Movies which have received the worst ratings             Rating
Movie_Id          
515       1.287879
3575      1.400000
2426      1.452381
4202      1.487395
1227      1.496241
